In [3]:
import boto3
dynamodb=boto3.resource('dynamodb')

table = dynamodb.create_table (
    TableName='ExchangeRatesSDK',
    KeySchema =[
            {
                'AttributeName': 'cc',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'exchangedate',
                'KeyType': 'RANGE'
            }
    ],
    AttributeDefinitions = [
            {
                'AttributeName': 'cc',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'exchangedate',
                'AttributeType': 'S'
            }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 5
    }
)
print(table)

dynamodb.Table(name='ExchangeRatesSDK')


In [1]:
import boto3
from pprint import pprint
dynamodb=boto3.resource('dynamodb')

table = dynamodb.Table('ExchangeRatesSDK')
response = table.put_item (
Item ={
    'txt': 'Австралійський долар',
    'rate': '21.6852',
    'cc': 'AUD',
    'exchangedate':'01.01.2021'
}
)
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 12 Nov 2022 13:23:08 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2745614147',
                                      'x-amzn-requestid': '4SFIQMPCQVCAQ6CHISGBMKVFK7VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4SFIQMPCQVCAQ6CHISGBMKVFK7VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [8]:
import boto3
from pprint import pprint 

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('ExchangeRatesSDK')

response = table.delete_item(Key={'cc':'AUD', 'exchangedate': '01.01.2021'})

pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Fri, 04 Nov 2022 20:47:29 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2745614147',
                                      'x-amzn-requestid': 'P1K7O59710P80V5MV25ME1E8BRVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'P1K7O59710P80V5MV25ME1E8BRVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [15]:
import boto3
import csv

s3 = boto3.client('s3')
items=[]
obj=s3.get_object(Bucket='cloudlaba2', Key = f'exchange.csv')
lines=obj['Body'].read().decode('utf-8').splitlines(True)
reader = csv.DictReader(lines)

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('ExchangeRatesSDK')
with table.batch_writer() as batch:
    for row in reader:
        if row['cc'] in ("EUR", "USD", "PLN"):
            item={}
            item['txt']=row['txt']
            item['rate']=row['rate']
            item['cc']=row['cc']
            item['exchangedate']=row['exchangedate']
            batch.put_item(Item=item)
        
print(batch)

In [33]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from pprint import pprint

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('ExchangeRatesSDK')

response = table.scan(FilterExpression=Attr('rate').gte('37'))

for item in response['Items']:
    pprint(item)
    
print("COUNT: ", len(response['Items']))

{'cc': 'EUR', 'exchangedate': '01.08.2022', 'rate': '37.3365', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '02.08.2022', 'rate': '37.4133', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '03.08.2022', 'rate': '37.3804', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '04.08.2022', 'rate': '37.2488', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '05.08.2022', 'rate': '37.2104', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '06.08.2022', 'rate': '37.2104', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '07.08.2022', 'rate': '37.2104', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '08.08.2022', 'rate': '37.417', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '09.08.2022', 'rate': '37.2963', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '10.08.2022', 'rate': '37.3804', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '11.08.2022', 'rate': '37.5102', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '12.08.2022', 'rate': '37.7516', 'txt': 'Євро'}
{'cc': 'EUR', 'exchangedate': '13.08.2022', 'rate': '37.7516', 't